# Graph Queries

<a href="https://colab.research.google.com/github/joerg84/Graph_Powered_ML_Workshop/blob/master/Graphs_Queries.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

First, setting up our environment.

In [ ]:
!git clone https://github.com/joerg84/Graph_Powered_ML_Workshop.git
!rsync -av Graph_Powered_ML_Workshop/ ./ --exclude=.git
!pip3 install pyarango

In [ ]:
import json
import requests
import sys
import oasis

from pyArango.connection import *
from pyArango.collection import Collection, Edges, Field
from pyArango.graph import Graph, EdgeDefinition
from pyArango.collection import BulkOperation as BulkOperation

First, create a temporary database:

In [ ]:
# Retrieve tmp credentials from ArangoDB Tutorial Service
login = oasis.getTempCredentials()

# Connect to your temp database
url = "https://"+login["hostname"]+":"+str(login["port"])
conn = Connection(arangoURL=url, username=login["username"], password=login["password"],)
db = conn[login["dbName"]] 

In [ ]:
print("https://"+login["hostname"]+":"+str(login["port"]))
print("Username: " + login["username"])
print("Password: " + login["password"])
print("Database: " + login["dbName"])

# First Graph

Let define a structure for a simple train network.

![trainNetwork](img/train_network.png)

In [ ]:
from pyArango.collection import Collection, Field
from pyArango.graph import Graph, EdgeDefinition


class Cities(Collection):
    _fields = {
        "country": Field()
    }

class Connection(Edges): 
    _fields = {
        "distance_km": Field(),
        "travel_time": Field()
    }

class RailNetwork(Graph) :
    _edgeDefinitions = [EdgeDefinition("Connection", fromCollections=["Cities"], toCollections=["Cities"])]
    _orphanedCollections = []

db.createCollection("Cities")
db.createCollection("Connection")
# same for the graph
railNetworkGraph = db.createGraph("RailNetwork")

# creating some documents
h1 = railNetworkGraph.createVertex('Cities', {"_key": "Berlin", "country" : "Germany"})
h2 = railNetworkGraph.createVertex('Cities', {"_key": "Boston", "country" : "USA"})

# linking them
e1 = railNetworkGraph.link('Connection', h1, h2, {"distance_km": 6077})

# Lookup berlin vertex
berlin = db["Cities"]["Berlin"] 

# As we unfortunately cannot travel from Berlin to Boston by train....
railNetworkGraph.unlink('Connection',berlin, h2)

In [ ]:

cities = [
    "Inverness",
    "Aberdeen",
    "Leuchars",
    "StAndrews",
    "Edinburgh",
    "Glasgow",
    "York",
    "Cologne",
    "Carlisle",
    "Birmingham",
    "London",
    "Brussels",
    "Toronto",
    "Winnipeg",
    "Saskatoon",
    "Edmonton",
    "Jasper",
    "Vancouver"
  ]

connections = [
    ( "Inverness", "Aberdeen", 3, 2.5 ),
    ( "Aberdeen", "Leuchars", 1.5, 1 ),
    ( "Leuchars", "Edinburgh", 1.5, 3 ),
    ( "Edinburgh", "Glasgow", 1, 1 ),
    ( "Edinburgh", "York", 3.5, 4 ),
    ( "Glasgow", "Carlisle", 1, 1 ),
    ( "Carlisle", "York", 2.5, 3.5 ),
    ( "Carlisle", "Birmingham", 2.0, 1 ),
    ( "Birmingham", "London", 1.5, 2.5 ),
    ( "Leuchars", "StAndrews", 0.2, 0.2 ),
    ( "York", "London", 1.8, 2.0 ),
    ( "London", "Brussels", 2.5, 3.5 ),
    ( "Brussels", "Cologne", 2, 1.5 ),
    ( "Toronto", "Winnipeg", 36, 35 ),
    ( "Winnipeg", "Saskatoon", 12, 5 ),
    ( "Saskatoon", "Edmonton", 12, 17 ),
    ( "Edmonton", "Jasper", 6, 5 ),
    ( "Jasper", "Vancouver", 12, 13 )
]

for city in cities:
    railNetworkGraph.createVertex('Cities', {"_key": city})

    
for city1, city2, time1, time2  in connections:
        # Note this is a directional graph 
        railNetworkGraph.link('Connection', db["Cities"][city1], db["Cities"][city2], {"travel_time": time1})
        railNetworkGraph.link('Connection', db["Cities"][city2], db["Cities"][city1], {"travel_time": time2})

print("Added Connections!")       

We can login into the database using the above login credentials and view the RailNetwork graph from the Graphs tab.

![trainNetwork](img/arango_train_graph.png)

While we are here, to understand the underlying format let us also visit the collections tab.

![trainNetwork](img/arango_collections.png)

Finally: Our first Graph Traversal

In [ ]:
reachabilty_query = """WITH Cities
FOR vertex, edge, path
  IN 1..5 
  OUTBOUND 'Cities/London'
  GRAPH 'RailNetwork'
  FILTER SUM(path.edges[*].travel_time) < 5
  return 
  { 'city': vertex._key,
    'path': CONCAT_SEPARATOR(" -> ", path.edges[*]._to)
  }"""

queryResult = db.AQLQuery(reachabilty_query, rawResults=True)
for result in queryResult:
    print("city: " + result["city"])
    print("path: Cities/London -> " + result["path"])
    print()

In [ ]:
Next: Shortest Path

In [ ]:
shortest_path_query = """FOR p IN OUTBOUND K_SHORTEST_PATHS 'Cities/Aberdeen' TO 'Cities/London'
  GRAPH 'RailNetwork'
      LIMIT 1
      RETURN {
          places: p.vertices[*]._key,
          travelTimes: p.edges[*].travel_time,
          travelTimeTotal: SUM(p.edges[*].travel_time)
      }"""

queryResult = db.AQLQuery(shortest_path_query, rawResults=True)
for result in  queryResult:
    print("places: " +  str(result['places']))
    print("intermediate travel times: " +  str(result['travelTimes']))
    print("total travel time: " +  str(result['travelTimeTotal']))
    print()

In [ ]:
# Alternative Shortest path query with more options
shortest_path_query = """FOR p IN OUTBOUND K_SHORTEST_PATHS'Cities/Aberdeen' TO 'Cities/London'
  GRAPH 'RailNetwork'
      OPTIONS {
      weightAttribute: "travel_time",
      defaultWeight: 100
      }
      LIMIT 3
      RETURN {
          places: p.vertices[*]._key,
          travelTimes: p.edges[*].travel_time,
          travelTimeTotal: SUM(p.edges[*].travel_time)
      }"""
queryResult = db.AQLQuery(shortest_path_query, rawResults=True)
for result in  queryResult:
    print("places: " +  str(result['places']))
    print("intermediate travel times: " +  str(result['travelTimes']))
    print("total travel time: " +  str(result['travelTimeTotal']))
    print()

In [ ]:
# Delete collections
db.dropAllCollections() 
db.reload()